In [8]:
import pandas as pd
from nltk import word_tokenize
from gensim.models import Word2Vec

/Users/jackoehling/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2.0 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:
google_df = pd.read_csv('processed_google_data.csv')
tweets_df = pd.read_csv('processed_cyberbullying_tweets.csv')

google_df.head()


,cyberbullying_type,processed_tweet_text
0,0,game hurt
1,0,right care fuck
2,0,man love reddit
3,0,name nowhere near falcon
4,0,right considering ’ important document know da...


In [3]:
tweets_df.head()

,cyberbullying_type,processed_tweet_text
0,0,words katandandre food crapilicious mkr
1,0,aussietv white mkr theblock imacelebrityau tod...
2,0,xochitlsuckks classy whore red velvet cupcakes
3,0,meh p thanks heads concerned another angry dud...
4,0,rudhoeenglish isis account pretending kurdish ...


In [5]:

combined_df = pd.concat([tweets_df, google_df], ignore_index=True)

FILE = 'processed_concat_data.csv'
combined_df.to_csv(FILE, index=False)



In [9]:
concat_df = pd.read_csv(FILE)

tweets = concat_df['processed_tweet_text'].astype(str).tolist()

tokenized_sentences = [word_tokenize(sentence.lower()) for sentence in tweets]
model_w2v = Word2Vec(sentences=tokenized_sentences, vector_size=100, window=5, min_count=1, workers=4)

model_w2v.wv['food']


vocab_size = len(model_w2v.wv.key_to_index)
vocab_size

concat_df['word_embeddings'] = concat_df.apply(lambda x : list(), axis=1)

for index, row in concat_df.iterrows():
    for word in word_tokenize(str(row['processed_tweet_text']).lower()):
        row['word_embeddings'].append(model_w2v.wv[word])
    while len(row['word_embeddings']) < 50:
        row['word_embeddings'].append([0]*100)
    

len(concat_df.loc[0, 'word_embeddings'])
len(concat_df.loc[0, 'word_embeddings'][0])


model_w2v.wv['words']

from sklearn.model_selection import train_test_split

X = concat_df['word_embeddings'].tolist()
y = concat_df['cyberbullying_type'].tolist()

data_0 = concat_df[concat_df['cyberbullying_type'] == 0]
data_1 = concat_df[concat_df['cyberbullying_type'] == 1]


191084


In [11]:
sampled_data_0 = data_0.sample(n=40000, random_state=42)
sampled_data_1 = data_1.sample(n=40000, random_state=42)

balanced_df = pd.concat([sampled_data_0, sampled_data_1])

X_dec = balanced_df['word_embeddings'].tolist()
y_dec = balanced_df['cyberbullying_type'].tolist()

# Shuffle the resulting DataFrame to mix the examples of 0s and 1s
balanced_df = balanced_df.sample(frac=1, random_state=42).reset_index(drop=True)

X_train, X_test, y_train, y_test = train_test_split(X_dec, y_dec, test_size=0.3, random_state=42)

In [12]:
import numpy as np
import tensorflow
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Bidirectional, LSTM, Dense, Dropout, Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences


In [13]:
# Padding sequences to ensure uniform input size
max_length = max(max(len(seq) for seq in X_train), max(len(seq) for seq in X_test))
X_train_padded = pad_sequences(X_train, maxlen=max_length, padding='post', dtype='float32')
X_test_padded = pad_sequences(X_test, maxlen=max_length, padding='post', dtype='float32')


In [14]:
model = Sequential()
model.add(Bidirectional(LSTM(64, return_sequences=True), input_shape=(max_length, 100)))
model.add(Bidirectional(LSTM(32)))
model.add(Dense(1, activation='sigmoid'))  # Use 'softmax' for multi-class classification

# Compiling the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])  # Use 'categorical_crossentropy' for multi-class classification

# Training the model
model.fit(X_train_padded, np.array(y_train), epochs=1, batch_size=64, validation_split=0.2)

# Evaluating the model
loss, accuracy = model.evaluate(X_test_padded, np.array(y_test))
print(f'Test Accuracy: {accuracy}')

/Users/jackoehling/Library/Python/3.9/lib/python/site-packages/keras/src/layers/core/wrapper.py:27: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


700/700 ━━━━━━━━━━━━━━━━━━━━ 217s 307ms/step - accuracy: 0.7563 - loss: 0.4820 - val_accuracy: 0.7893 - val_loss: 0.4314
750/750 ━━━━━━━━━━━━━━━━━━━━ 53s 71ms/step - accuracy: 0.7965 - loss: 0.4211
Test Accuracy: 0.7952916622161865


In [43]:
def predict_sentiment(sentence, model, word2vec_model):
    # Tokenize and convert to embeddings
    tokens = word_tokenize(sentence.lower())
    print(tokens)
    embeddings = [word2vec_model.wv[token] if token in word2vec_model.wv else np.zeros(100) for token in tokens]
    
    # Pad the sequence
    padded_embeddings = pad_sequences([embeddings], maxlen=max_length, padding='post', dtype='float32')
    print(padded_embeddings)
    # Predict
    prediction = model.predict(padded_embeddings)
    print(prediction)
    # Assuming binary classification with a sigmoid output layer
    predicted_label = 'Positive' if prediction[0][0] < 0.50 else 'Negative'
    
    return predicted_label, prediction[0][0]

# Example usage
test_sentence = "ur so freaking ugly"
predicted_label, confidence = predict_sentiment(test_sentence, model, model_w2v)

print(f"Sentence: '{test_sentence}'")
print(f"Predicted Sentiment: {predicted_label} (Confidence: {confidence:.2f})")

['ur', 'so', 'freaking', 'ugly']
[[[ 2.3538692   0.5744179   0.1304177  ...  0.21989945 -2.1070566
    0.55537146]
  [ 0.          0.          0.         ...  0.          0.
    0.        ]
  [ 0.20748849  0.2607715   0.38720724 ... -0.47075617  0.0126165
    0.01858659]
  ...
  [ 0.          0.          0.         ...  0.          0.
    0.        ]
  [ 0.          0.          0.         ...  0.          0.
    0.        ]
  [ 0.          0.          0.         ...  0.          0.
    0.        ]]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
[[0.7973456]]
Sentence: 'ur so freaking ugly'
Predicted Sentiment: Negative (Confidence: 0.80)
